In [1]:
import requests
from Tidy_Data_new import *

In [ ]:
if not os.path.exists("csv"):
    os.makedirs("csv")
for year in [2020,2021,2022]: # 2020 has been excluded to be reserved as the test set
    read_a_season("../nhl_data/",year).to_csv('csv/tidy_{season}.csv'.format(season = year), sep = ',', index = False)

In [4]:
directory = 'csv'

# Get a list of csv file names within the directory
csv_files = [f for f in os.listdir(directory) if f.endswith('.csv')]

# Sort the list of csv files
csv_files.sort()

# List to hold your DataFrames
dataframes_list = []

# Loop over the sorted list of csv files with tqdm for progress indication
for filename in tqdm(csv_files, desc="Loading files", unit="file"):
    file_path = os.path.join(directory, filename)
    df = pd.read_csv(file_path)

    # Filter the DataFrame to only include rows where 5th and 6th digits of 'game_id' are '02' (Regular Season)

    dataframes_list.append(df)

# Concatenate all the DataFrames in the list into a single DataFrame
test_df = pd.concat(dataframes_list, ignore_index=True)

# Sort the DataFrame first by 'game_id' and then by 'event_idx'
test_df= test_df.sort_values(by=['game_id', 'event_idx'])

# Reset the index of the sorted DataFrame
test_df = test_df.reset_index(drop=True)

Loading files: 100%|██████████| 3/3 [00:00<00:00,  5.28file/s]


In [5]:
test_df

,game_id,event_idx,play_type,shot_type,shot_dist,game_time,period,period_time_rem,coordinate,shooter_id,goalie_id,empty_Net,rink_side,angle_net,is_goal
0,2020020001,9,Shot,Wrist Shot,47.169906,2021-01-13T22:30:00Z,1,19:26,"{'x': 49, 'y': -25}",8478542,8479394.0,0,left,-32.005383,0
1,2020020001,34,Shot,Snap Shot,63.812225,2021-01-13T22:30:00Z,1,08:21,"{'x': -35, 'y': -34}",8477462,8477465.0,0,right,32.195734,0
2,2020020001,42,Shot,Wrist Shot,29.068884,2021-01-13T22:30:00Z,1,04:18,"{'x': 87, 'y': 29}",8471675,8479394.0,0,left,86.054814,0
3,2020020001,53,Shot,Wrist Shot,32.649655,2021-01-13T22:30:00Z,1,19:44,"{'x': -74, 'y': 29}",8478439,8477465.0,0,right,-62.650124,0
4,2020020001,55,Shot,Backhand,48.270074,2021-01-13T22:30:00Z,1,18:55,"{'x': -52, 'y': -31}",8480797,8477465.0,0,right,39.957549,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
324175,2022030415,796,Shot,Wrist Shot,126.826653,2023-06-14T00:00:00Z,3,02:01,"{'x': -32, 'y': 38}",8479372,8478499.0,0,left,17.434875,0
324176,2022030415,1001,Shot,Wrist Shot,61.400326,2023-06-14T00:00:00Z,3,01:05,"{'x': -36, 'y': -31}",8477447,8475683.0,0,right,30.323607,0
324177,2022030415,1002,Goal,Wrist Shot,5.830952,2023-06-14T00:00:00Z,3,01:02,"{'x': -86, 'y': 5}",8478462,8475683.0,0,right,-59.036243,1
324178,2022030415,1055,Shot,Wrist Shot,6.403124,2023-06-14T00:00:00Z,3,05:31,"{'x': 84, 'y': 4}",8479066,8478499.0,0,left,38.659808,0


In [100]:
r = requests.get(
	"http://localhost:8080/check", 
)
r.json()

['shot_dist']

In [2]:
r = requests.get(
	"http://localhost:8080/logs", 
)
r.json()

NameError: name 'requests' is not defined

In [3]:
request1 = {"model_name":"log_reg_shot_dist_and_angle",
           "version":"1.9.0"}

request2 = {"model_name":"Log_Reg_shot_dist_only",
           "version":"1.17.0"}


request3 = {"model_name":"qwfqwfqwf",
           "version":"1.17.0"}

# request.get_json()
r = requests.post(
	"http://localhost:8080/download_registry_model", 
    json= request2
)
r.json()

'Log_Reg_shot_dist_only is not a valide model'

In [14]:
import joblib
from flask import Flask, jsonify, request, abort
model = joblib.load('serving/models/Log_Reg_shot_dist_only.joblib')
model

LogisticRegression()

In [15]:
input = test_df.loc[1:100][['shot_dist','angle_net']]
r= requests.post(
    "http://localhost:8080/predict",
    json =input.to_json()
)
# r.json()

In [16]:
r.json()

'{"chance of goal":{"0":0.0330831896,"1":0.0975061246,"2":0.0855079663,"3":0.0539783583,"4":0.0370185508,"5":0.0442582565,"6":0.075899741,"7":0.1282438058,"8":0.0568594851,"9":0.067498384,"10":0.1676463238,"11":0.0540635597,"12":0.1439588429,"13":0.0646273808,"14":0.1180440447,"15":0.0583658374,"16":0.0373138366,"17":0.0605328246,"18":0.1250413385,"19":0.0531495274,"20":0.1652595431,"21":0.0379554922,"22":0.1153025186,"23":0.1396280803,"24":0.0483641135,"25":0.0566231776,"26":0.0554652628,"27":0.0089415464,"28":0.0928969571,"29":0.04080061,"30":0.036955006,"31":0.1618675555,"32":0.0883088991,"33":0.1860142774,"34":0.1518443206,"35":0.0608770831,"36":0.0126809505,"37":0.1020621768,"38":0.0736265818,"39":0.0208033626,"40":0.069617514,"41":0.1304917389,"42":0.1610541394,"43":0.0596350031,"44":0.01896693,"45":0.0569617429,"46":0.0564868697,"47":0.0542306724,"48":0.1780279023,"49":0.0528957438,"50":0.1153025186,"51":0.0218453829,"52":0.0521716748,"53":0.0481293084,"54":0.0891366503,"55":0.0